# 7. 코딩 테스트에 나오는 수학

- 코딩 테스트에 나오는 수학은 **정수론** 그리고 **기하**가 대표적이다.
- 코딩 테스트 내의 기하
  - 피타고라스의 정리를 활용한 점 사이의 거리 등을 쉽게 알 수 있는 내용
  - CCW, 컨벡스헐, 좌표기하 등 내용이 어려워 난이도 격차가 꽤 있는 분야

- 다뤄볼 주제
  - gcd, lcm
  - 소수 체크와 소인수 분해
  - **에라토스테네스의 체 활용**
  - 거듭제곱 연산

<br>

## 7.1 GCD와 LCM

- GCD : Greatest Common Divider, 최대 공약수
- LCM : Least Common Multiple, 최소 공배수  
  
  
- 최대 공약수 / 최소 공배수 문제의 90% 이상은 **유클리드 호제법 알고리즘**을 사용한다.
- 최소공배수(LCM)의 경우에는 다음과 같은 식으로 풀 수 있으므로 최대공약수(GCD)만 알면 된다.

$$
LCM(a,b) = \frac{a \times b}{GCD(a,b)}
$$

- GCD, LCM 문제 해결 방법
  - 단순 반복문으로 하는 방법
  - 유클리드 호제법을 이용한 방법
  - 라이브러리를 사용하는 방법

- 유클리드 호제법의 경우는 다음 성질을 활용한 방법이다.

$$
GCD(a,b) = GCD(b, a\%b)
$$

- a와 b의 최대공약수는 b와 a/b의 나머지의 최대공약수와 같다.

In [1]:
# 방법 1 : 단순 반복문 사용
#  - 시간 복잡도 : O(n)
def gcd_naive(a, b):
    for i in range(min(a,b),0,-1):
        if a % i == 0 and b % i == 0: return i

In [23]:
%time
print(gcd_naive(100000000, 2*30))

Wall time: 0 ns
20


In [9]:
# 방법 2-1 : 유클리드 호제법 이용
def gcd(a, b):
    if a % b == 0: return b # a/b의 나머지가 0일 떄 a 와 b 중 작은 수가 최대공약수이다.
    return gcd(b, a%b)

In [10]:
def gcd(a, b):
    return gcd(b, a%b) if a%b != 0 else b

In [24]:
%time
print(gcd(100000000, 2*30))

Wall time: 0 ns
20


In [5]:
# 방법 2-2 : 반복문으로 변경
def gcd2(a, b):
    while a % b != 0: a, b = b, a%b
    return b

In [27]:
%time
print(gcd2(100000000, 2*30))

Wall time: 0 ns
20


In [31]:
# 방법 3 : math의 gcd 사용
#  - 이 방법이 속도가 가장 빠르다
import math

print(math.gcd(1,2))

1


In [28]:
%time
print(math.gcd(100000000, 2*30))

Wall time: 0 ns
20


<br>

- LCM은 `gcd`를 활용하여 계산한다.
- Python이 아닌 다른 언어의 경우 int overflow가 발생할 수 있다.
- 그러므로 `a` / `gcd(a,b)` * `b` 순으로 하는 것을 추천한다

In [14]:
def lcm(a, b):
    return int(a*b/gcd(a,b))

In [15]:
print(lcm(10, 24))

120


<br>

## 7.2 소수 체크와 소인수 분해

- 소수 체크와 소인수 분해도 많이 나오는 유형이다.
- 소수 체크 : 반복문으로 진행
- 소인수 분해 : 약간의 트릭 활용
- 두 알고리즘의 시간 복잡도는 모두 $O\left(\sqrt{N}\right)$ 이다.

<br>

### 7.2.1 소수 체크

In [32]:
# 소수 체크 기본
# (prime_check 말고 isPrime 등의 관용적인 함수명을 사용)
def isPrime(N):
    for i in range(2, N):
        if N%i == 0: return False # 나눠지면 해당 수는 소수가 아님
        if i*i > N: break # i가 루트 N 보다 커지면 리턴 (시간 효율성을 위한 로직)
    return True

In [33]:
print(isPrime(3))

True


In [34]:
print(isPrime(6))

False


<br>

### 7.2.2 소인수 분해

In [36]:
# 소인수분해 기본
def prime_factorization(N):
    p, fac = 2, []
    while p**2 <= N: # 시간 효율성을 높이기 위한 로직
        if N%p == 0:
            N //= p # N = N/p
            fac.append(p)
        else:
            p += 1
    if N > 1: fac.append(N)
    return fac

In [37]:
print(prime_factorization(12))

[2, 2, 3]


In [38]:
print(prime_factorization(12345))

[3, 5, 823]


<br>

### 7.2.3 에라토스테네스의 체를 활용한 소수 리스트 구하기

- 이런 알고리즘이 단 한 번 사용하거나 빠르게 체크할 때는 좋다.
- 하지만 여러 쿼리를 묻는 문제 등의 경우에는 시간 복잡도가 꽤 크다.
- 이런 문제를 해결하기 위해 소수 리스트를 미리 만드는 방법이 있는데 이것이 **에라토스테네스의 체** 이다.

In [43]:
# 에라토스테네스의 체를 활용한 소수 리스트 구하기
# - 0 ~ N 까지의 숫자 중 소수 리스트 구하기
# - 작은 수 부터 해당 숫자가 소수인 지 확인
# - 해당 숫자가 소수이면 그 숫자의 배수들을 모두 제거
# - 다음 숫자부터 위 과정 반복
def era_prime(N):
    A, p = [0 for _ in range(N+1)], []
    for i in range(2, N):
        if A[i] == 0: p.append(i) # A 에 체크가 안되어 있다. -> 합성수가 아니다 = 소수이다.
        else: continue
        for j in range(i**2, N, i):
            A[j] = 1
    return p

In [44]:
print(era_prime(12))

[2, 3, 5, 7, 11]


In [45]:
print(era_prime(100))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


<br>

- 이런 소수 리스트를 만들면 소인수분해도 다음과 같이 할 수 있다.

In [48]:
# 소수 리스트가 있는 경우 소인수분해
# 소수 리스트의 크기는 sqrt(N) 보다 커야 함
def prime_factorization_2(N, p):
    fac = []
    for i in p:
        if N == 1 or N > i*i: break
        while N%i == 0:
            fac.append(i)
            N //= 0
    return fac

<br>

## 7.3 빠른 거듭제곱과 모듈러

- Python에서는 크게 고민할 부분은 아니지만 거듭제곱 연산을 해야 할 때가 있다.
- 이런 거듭제곱을 순수하게 반복문으로 진행하는 것이 아니라 효율적으로 할 수 있는 방법이 있다.
  - $a^{11} = a^{1} \times a^{2} \times a^{8}$

In [54]:
def pow_advanced(a, b, mod):
    ret = 1
    while b > 0:
        if b % 2: ret = ret*a%mod
        a, b = a*a%mod, b//2
    return ret

%time pow_advanced(3, 10000000, 1000000007)
%time pow(3, 10000000, 1000000007)
%time 3**10000000%1000000007

Wall time: 0 ns
Wall time: 0 ns
Wall time: 3.41 s


769346453

In [50]:
print(prime_factorization_2(100, era_prime(100)))

[]
